![banner](../banner.jpg)

***Training course in data analysis for genomic surveillance of African malaria vectors***

---

# Exploration of the _Anopheles funestus_ resource

**Theme: Data**

This module provides an introduction to accessing and exploring data about *Anopheles funestus* mosquito specimens collected in the field and submitted for whole-genome sequencing by MalariaGEN.

A certain degree of familiarity with the content of the training course on _Anopheles gambiae s.l._ is expected. Many of the concepts presented in this module were introduced in the training course and we will refer to the relevant workshop and module instead of giving detailed explanations.

## Learning objectives

After completing this module, you will be able to:

* Use the `malariagen_data` Python package to access `Af1.0` data in Google Cloud.
* Explore the `Af1.0` data release and summarise the mosquito samples for which genomic data are available using pivot tables and maps.
* Use genomic coordinates to identify sites and regions within the reference genome.
* Find specific genes of interest in the reference genome.

## Lecture?

### English

### Français

Please note that the code in the cells below might differ from that shown in the video. This can happen because Python packages and their dependencies change due to updates, necessitating tweaks to the code.

## Why are we interested in _An. funestus_ ?

In [2]:
%%html
<img width="50%" height="50%" src="https://github.com/anopheles-genomic-surveillance/anopheles-genomic-surveillance.github.io/blob/master/src/img/advanced/MAP_Africa_vector_range.png?raw=1"/>

An. funestus is a major vector of malaria in sub-Saharan Africa, particularly in East Africa. Its range covers most of the continent, as shown on the map above courtesy of the Malaria Atlas Project [1]. If vector control interventions are successful in reducing the populations of _An. gambiae s.l._ withour affecting as much the _An. funestus_ populations, they could lead to _An. funestus_ replacing _An. gambiae s.l._ as a malaria vector. This has, for instance, already been observed in East Africa. Msugupakulya et al. [2] have shown that, while _An. gambiae s.l._ was the main vector of malaria in East Africa during the first decade of the XXI<sup>st</sup> century, _An. funestus_ has taken that role more recently.

_An. funestus_ is known to be highly anthropophilic [3]. It also been shown to have a significantly longer lifespan than other African malaria vectors [4]. In some areas, it is also reported to have very high Plasmodium infection rates [5, 6, 7]. _An funestus_ has adapted recently to the use of indoor-based interventions, including bed nets, by modifying its behaviour. It is now known to be probe to outdoor, early evening and late morning biting [8, 9, 10]. There is also evidence for a preference for larval habitats that persist in the dry months [11] which may extend the transmission season. 

## Why is this a different course from the one on _An. gambiae s.l._?

All the methods that were introduced in the _An. gambiae s.l._ course to analyse population structure ([Workshop 3](https://anopheles-genomic-surveillance.github.io/workshop-3/about.html)), genetic diversity ([Workshop 5](https://anopheles-genomic-surveillance.github.io/workshop-5/about.html)), positive selection ([Workshop 6](https://anopheles-genomic-surveillance.github.io/workshop-6/about.html)), gene flow ([Workshop 7](https://anopheles-genomic-surveillance.github.io/workshop-7/about.html)) or insecticide resistance ([Workshop 1](https://anopheles-genomic-surveillance.github.io/workshop-1/about.html) and [Workshop 2](https://anopheles-genomic-surveillance.github.io/workshop-2/about.html)) can be applied to _An. funestus_ (and will be in [Advanced funestus - Analyses](https://anopheles-genomic-surveillance.github.io/advanced-training-materials/funestus_analyses.html)) but, particularly for insecticide resistance analyses, the fact that _An. gambiae s.l._ and _An. funestus_ are two very different species cannot be ignored.

To begin with, there is no gene flow between _An. gambiae s.l._ and _An. funestus_. The two species have common ancestors, they do belong to the same genus _Anopheles_, but they diverged around 9.5 million years ago [12]. This, obviously, predates any human intervention as it predates humanity [13]. As a result, any insecticide resistance mutation that is observed will either be different or a co-evolution.

The `Af1.0` resource is built using the `idAnoFuneDA-416_04` [14] genome assembly. As we will see, while the _An. funestus_ genome has the same number of chromosomes (2 autosomes and X) as the _An. gambiae s.l._, they are of different lengths, the inversions are different and the genes of interest may be located in completely different places in the genome. 

In [4]:
%%html
<img width="50%" height="50%" src="https://github.com/anopheles-genomic-surveillance/anopheles-genomic-surveillance.github.io/blob/master/src/img/advanced/funestus_synteny.png?raw=1"/>
#I need to find a good picture to put here

## Accessing the `Af1.0` data resource

We are now going to be accessing and analysing data from the [*Anopheles funestus* 1000 Genomes Project phase 1 data resource](https://malariagen.github.io/vector-data/af1/af1.0.html), also known as "Af1.0" for short. This includes data from whole-genome sequencing of 656 mosquitoes from 13 African countries.

To set up your notebook to access these data, first install the [malariagen_data](https://github.com/malariagen/malariagen-data-python) package.

In [1]:
%pip install -q --no-warn-conflicts malariagen_data

We will also need to use ipyleaflet to make some maps

In [2]:
%pip install -qq ipyleaflet

In [3]:
import malariagen_data
import ipyleaflet

In [4]:
af1 = malariagen_data.Af1()
af1

<MalariaGEN Af1 API client>
Storage URL             : gs://vo_afun_release_master_us_central1
Data releases available : 1.0, 1.1, 1.2, 1.3, 1.4
Results cache           : None
Cohorts analysis        : 20240515
Site filters analysis   : dt_20200416
Software version        : malariagen_data 15.2.0
Client location         : Iowa, United States (Google Cloud us-central1)
---
Please note that data are subject to terms of use,
for more information see https://www.malariagen.net/data
or contact support@malariagen.net. For API documentation see 
https://malariagen.github.io/malariagen-data-python/v15.2.0/Af1.html

## Accessing and exploring *Anopheles funestus* genomic data

### Loading sample metadata

We can use the [`sample_metadata()`](https://malariagen.github.io/malariagen-data-python/latest/Af1.html#sample-metadata-access) function to retrieve a pandas DataFrame containing metadata about all 656 samples in the Af1.0 resource. In this DataFrame, each row represents one mosquito sample, and the columns such as `country` and `year` provide information about where the mosquito was originally collected.

In [5]:
df_samples = af1.sample_metadata(sample_sets="1.0")
df_samples

,sample_id,partner_sample_id,contributor,country,location,year,month,latitude,longitude,sex_call,...,admin1_name,admin1_iso,admin2_name,taxon,cohort_admin1_year,cohort_admin1_month,cohort_admin1_quarter,cohort_admin2_year,cohort_admin2_month,cohort_admin2_quarter
0,VBS24195,1229-GH-A-GH01,Samuel Dadzie,Ghana,Dimabi,2017,8,9.420,-1.083,F,...,Northern Region,GH-NP,Tolon,funestus,GH-NP_fune_2017,GH-NP_fune_2017_08,GH-NP_fune_2017_Q3,GH-NP_Tolon_fune_2017,GH-NP_Tolon_fune_2017_08,GH-NP_Tolon_fune_2017_Q3
1,VBS24196,1229-GH-A-GH02,Samuel Dadzie,Ghana,Gbullung,2017,7,9.488,-1.009,F,...,Northern Region,GH-NP,Kumbungu,funestus,GH-NP_fune_2017,GH-NP_fune_2017_07,GH-NP_fune_2017_Q3,GH-NP_Kumbungu_fune_2017,GH-NP_Kumbungu_fune_2017_07,GH-NP_Kumbungu_fune_2017_Q3
2,VBS24197,1229-GH-A-GH03,Samuel Dadzie,Ghana,Dimabi,2017,7,9.420,-1.083,F,...,Northern Region,GH-NP,Tolon,funestus,GH-NP_fune_2017,GH-NP_fune_2017_07,GH-NP_fune_2017_Q3,GH-NP_Tolon_fune_2017,GH-NP_Tolon_fune_2017_07,GH-NP_Tolon_fune_2017_Q3
3,VBS24198,1229-GH-A-GH04,Samuel Dadzie,Ghana,Dimabi,2017,8,9.420,-1.083,F,...,Northern Region,GH-NP,Tolon,funestus,GH-NP_fune_2017,GH-NP_fune_2017_08,GH-NP_fune_2017_Q3,GH-NP_Tolon_fune_2017,GH-NP_Tolon_fune_2017_08,GH-NP_Tolon_fune_2017_Q3
4,VBS24199,1229-GH-A-GH05,Samuel Dadzie,Ghana,Gupanarigu,2017,8,9.497,-0.952,F,...,Northern Region,GH-NP,Kumbungu,funestus,GH-NP_fune_2017,GH-NP_fune_2017_08,GH-NP_fune_2017_Q3,GH-NP_Kumbungu_fune_2017,GH-NP_Kumbungu_fune_2017_08,GH-NP_Kumbungu_fune_2017_Q3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
651,VBS24534,1240-MZ-A-MozF_1314,Lizette Koekemoer,Mozambique,Motinho,2015,8,-10.851,40.594,F,...,Cabo Delgado,MZ-P,Palma,funestus,MZ-P_fune_2015,MZ-P_fune_2015_08,MZ-P_fune_2015_Q3,MZ-P_Palma_fune_2015,MZ-P_Palma_fune_2015_08,MZ-P_Palma_fune_2015_Q3
652,VBS24535,1240-MZ-A-MozF_1315,Lizette Koekemoer,Mozambique,Motinho,2015,8,-10.851,40.594,F,...,Cabo Delgado,MZ-P,Palma,funestus,MZ-P_fune_2015,MZ-P_fune_2015_08,MZ-P_fune_2015_Q3,MZ-P_Palma_fune_2015,MZ-P_Palma_fune_2015_08,MZ-P_Palma_fune_2015_Q3
653,VBS24536,1240-MZ-A-MozF_1317,Lizette Koekemoer,Mozambique,Motinho,2015,8,-10.851,40.594,F,...,Cabo Delgado,MZ-P,Palma,funestus,MZ-P_fune_2015,MZ-P_fune_2015_08,MZ-P_fune_2015_Q3,MZ-P_Palma_fune_2015,MZ-P_Palma_fune_2015_08,MZ-P_Palma_fune_2015_Q3
654,VBS24537,1240-MZ-A-MozF_1319,Lizette Koekemoer,Mozambique,Motinho,2015,8,-10.851,40.594,F,...,Cabo Delgado,MZ-P,Palma,funestus,MZ-P_fune_2015,MZ-P_fune_2015_08,MZ-P_fune_2015_Q3,MZ-P_Palma_fune_2015,MZ-P_Palma_fune_2015_08,MZ-P_Palma_fune_2015_Q3


We can then group these samples according to country and year of collection, sample set, and taxon.

In [6]:
df_samples.groupby(["country", "year", "sample_set", "taxon"]).size()

country                           year  sample_set                     taxon   
Benin                             2014  1231-VO-MULTI-WONDJI-VMF00043  funestus    37
Cameroon                          2014  1231-VO-MULTI-WONDJI-VMF00043  funestus    45
Central African Republic          2016  1230-VO-GA-CF-AYALA-VMF00045   funestus    10
Democratic Republic of the Congo  2015  1231-VO-MULTI-WONDJI-VMF00043  funestus    34
                                  2017  1240-VO-CD-KOEKEMOER-VMF00099  funestus    43
Gabon                             2017  1230-VO-GA-CF-AYALA-VMF00045   funestus    40
Ghana                             2014  1231-VO-MULTI-WONDJI-VMF00043  funestus    31
                                  2017  1229-VO-GH-DADZIE-VMF00095     funestus    36
Kenya                             2014  1232-VO-KE-OCHOMO-VMF00044     funestus    37
                                  2016  1232-VO-KE-OCHOMO-VMF00044     funestus    44
Malawi                            2014  1231-VO-MULTI-WONDJI

As we can see, all samples were collected between 2014 and 2018 and all samples were identified as _An. funestus_. We will thus not be able to look at other members of the complex (e.g., _An. vaneedeni_). We can also see that most countries are only represented by one sample set.

We can represent the provenance of the samples in a pivot table to better see the temporal distribution of samples.

In [7]:
pivot_location_year = (
    df_samples
    .pivot_table(
        index=["country", "location", "latitude", "longitude"],
        columns=["year"],
        values="sample_id",
        aggfunc="count",
        fill_value=0,
    )
)

pivot_location_year

year                                                                2014  \
country                          location       latitude longitude         
Benin                            Kpome           6.383    2.200       37   
Cameroon                         Mibellon        6.083    11.500      45   
Central African Republic         Bangui          4.377    18.608       0   
Democratic Republic of the Congo Kinshasa       -4.327    15.313       0   
                                 Watsa Moke      3.094    29.548       0   
Gabon                            Benguia        -1.633    13.492       0   
Ghana                            Dimabi          9.420   -1.083        0   
                                 Gbullung        9.488   -1.009        0   
                                 Gupanarigu      9.497   -0.952        0   
                                 Obuasi          5.933   -1.617       31   
                                 Woribogu        8.869    0.321        0   
Kenya                            Ahero          -0.174    34.920      37   
                                 Awendo         -0.901    34.533       0   
                                 Nyamilu        -1.074    34.391       0   
                                 Sumba          -0.686    34.596       0   
                                 Uradi           0.980    34.543       0   
                                 Uriri          -0.952    34.513       0   
Malawi                           Chikwawa       -16.017   34.789      18   
Mozambique                       Motinho        -10.851   40.594       0   
                                 Palmeira       -25.255   32.873       0   
                                 Palmeiras      -25.265   32.877       0   
Nigeria                          Akaka Remo      6.950    3.717        0   
Tanzania                         Minepa Village -8.273    36.681       0   
Uganda                           Tororo          0.750    34.083      49   
Zambia                           Katete         -14.183   31.867       0   

year                                                                2015  \
country                          location       latitude longitude         
Benin                            Kpome           6.383    2.200        0   
Cameroon                         Mibellon        6.083    11.500       0   
Central African Republic         Bangui          4.377    18.608       0   
Democratic Republic of the Congo Kinshasa       -4.327    15.313      34   
                                 Watsa Moke      3.094    29.548       0   
Gabon                            Benguia        -1.633    13.492       0   
Ghana                            Dimabi          9.420   -1.083        0   
                                 Gbullung        9.488   -1.009        0   
                                 Gupanarigu      9.497   -0.952        0   
                                 Obuasi          5.933   -1.617        0   
                                 Woribogu        8.869    0.321        0   
Kenya                            Ahero          -0.174    34.920       0   
                                 Awendo         -0.901    34.533       0   
                                 Nyamilu        -1.074    34.391       0   
                                 Sumba          -0.686    34.596       0   
                                 Uradi           0.980    34.543       0   
                                 Uriri          -0.952    34.513       0   
Malawi                           Chikwawa       -16.017   34.789       0   
Mozambique                       Motinho        -10.851   40.594      40   
                                 Palmeira       -25.255   32.873       0   
                                 Palmeiras      -25.265   32.877       0   
Nigeria                          Akaka Remo      6.950    3.717       41   
Tanzania                         Minepa Village -8.273    36.681       0   
Uganda                           Tororo        

Again, we can observe that most countries are only represented by a single location.

Finally, we can represent the origin of these samples on a map for a more visual interpretation of their distribution.

In [8]:
# create a map
m = ipyleaflet.Map(
    basemap=ipyleaflet.basemaps.OpenStreetMap.Mapnik,
    center=[0, 20],
    zoom=3,
)

# add markers for sampling locations
for row in pivot_location_year.reset_index().itertuples():
    title = f"{row.location}, {row.country} ({row.latitude:.3f}, {row.longitude:.3f})"
    for i, y in enumerate(pivot_location_year.columns):
      r = list(pivot_location_year.loc[(row.country, row.location, row.latitude, row.longitude)])
      if r[i] != 0:
        title += "\n" + str(r[i]) + " samples in " + str(y)
    marker = ipyleaflet.Marker(
        location=(row.latitude, row.longitude),
        draggable=False,
        title=title,
    )
    m.add_layer(marker)

# add a scale bar
m.add_control(ipyleaflet.ScaleControl(position="bottomleft"))

# display the map
m

Map(center=[0, 20], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_text…

## Exploring the reference and finding genes of interest

We saw during the third module of the first workshop of the course on _An. gambiae_ ([Workshop 1 Module 2](https://anopheles-genomic-surveillance.github.io/workshop-1/module-3-genome.html)) how to access the genome annotations for that species. The same function exists for _An. funestus_.

In [9]:
af1.genome_features()

,contig,source,type,start,end,score,strand,phase,ID,Parent,Note,description
0,2RL,VEuPathDB,ncRNA_gene,66492,69180,NaN,+,NaN,LOC125765358,NaN,NaN,"uncharacterized LOC125765358, transcript varia..."
1,2RL,VEuPathDB,lnc_RNA,66492,69180,NaN,+,NaN,LOC125765358_t2,LOC125765358,NaN,"uncharacterized LOC125765358, transcript varia..."
2,2RL,VEuPathDB,lnc_RNA,66492,69180,NaN,+,NaN,LOC125765358_t1,LOC125765358,NaN,"uncharacterized LOC125765358, transcript varia..."
3,2RL,VEuPathDB,exon,66492,66764,NaN,+,NaN,exon_LOC125765358_t2-E1,"LOC125765358_t2,LOC125765358_t1",NaN,NaN
4,2RL,VEuPathDB,lnc_RNA,66923,69180,NaN,+,NaN,LOC125765358_t3,LOC125765358,NaN,"uncharacterized LOC125765358, transcript varia..."
...,...,...,...,...,...,...,...,...,...,...,...,...
347419,X,VEuPathDB,exon,22261674,22261945,NaN,-,NaN,exon_LOC125771994_t1-E5,LOC125771994_t1,NaN,NaN
347420,X,VEuPathDB,exon,22262762,22262829,NaN,-,NaN,exon_LOC125771994_t1-E4,LOC125771994_t1,NaN,NaN
347421,X,VEuPathDB,exon,22263509,22263916,NaN,-,NaN,exon_LOC125771994_t1-E3,LOC125771994_t1,NaN,NaN
347422,X,VEuPathDB,exon,22263985,22264052,NaN,-,NaN,exon_LOC125771994_t1-E2,LOC125771994_t1,NaN,NaN


The first big difference between the reference for _An. funestus_ and _An. gambiae_ is that contigs are whole chromosomes (e.g., "2RL") instead of chromosome arms (e.g., "2R").

In [10]:
af1.genome_features().groupby("contig").size()

contig
2RL    170850
3RL    141790
X       34784
dtype: int64

We can group the annotations by "type" to see what the annotations contain.

In [11]:
af1.genome_features().groupby("type").size()

type
CDS                       161215
exon                       78083
five_prime_UTR             42054
lnc_RNA                      942
mRNA                       25010
ncRNA                        609
ncRNA_gene                  1041
protein_coding_gene        11962
pseudogene                    42
pseudogenic_transcript        42
rRNA                         101
snRNA                         27
snoRNA                         9
tRNA                         316
three_prime_UTR            25971
dtype: int64

Many different types of annotations are provided but we will only be interested in "protein_coding_gene"s for this module.

In [12]:
af1.genome_features().query("type == 'protein_coding_gene'")

,contig,source,type,start,end,score,strand,phase,ID,Parent,Note,description
16,2RL,VEuPathDB,protein_coding_gene,81055,82221,NaN,+,NaN,LOC125766088,NaN,NaN,60S ribosomal protein L18 [Source:UniProtKB/Tr...
26,2RL,VEuPathDB,protein_coding_gene,82584,84996,NaN,-,NaN,LOC125766087,NaN,NaN,Folliculin [Source:UniProtKB/TrEMBL;Acc:A0A182...
40,2RL,VEuPathDB,protein_coding_gene,141143,146973,NaN,+,NaN,LOC125766196,NaN,NaN,uncharacterized LOC125766196
50,2RL,VEuPathDB,protein_coding_gene,159192,185030,NaN,-,NaN,LOC125765747,NaN,NaN,"lachesin-like, transcript variant X5"
142,2RL,VEuPathDB,protein_coding_gene,227643,228822,NaN,+,NaN,LOC125760444,NaN,NaN,actin-related protein 2/3 complex subunit 4
...,...,...,...,...,...,...,...,...,...,...,...,...
347372,X,VEuPathDB,protein_coding_gene,22057024,22057530,NaN,-,NaN,LOC125768238,NaN,NaN,transcriptional regulatory protein AlgP-like
347378,X,VEuPathDB,protein_coding_gene,22078120,22083556,NaN,+,NaN,LOC125769511,NaN,NaN,uncharacterized LOC125769511
347386,X,VEuPathDB,protein_coding_gene,22089764,22093137,NaN,-,NaN,LOC125768244,NaN,NaN,uncharacterized LOC125768244
347394,X,VEuPathDB,protein_coding_gene,22192285,22193524,NaN,+,NaN,LOC125768250,NaN,NaN,protein lag-3-like


We can see that the naming convention ("LOC?????????") is different from the one used for _An. gambiae_ ("AGAP??????").

In order to locate genes of interest, several methods can be used. The easiest is when the ID is already known. For instance, in supplementary Table 3 of [15], the ID for GSTE2 is given as "LOC125763977". We can thus look at this gene.

In [13]:
af1.genome_features().query("ID == 'LOC125763977'")

,contig,source,type,start,end,score,strand,phase,ID,Parent,Note,description
133312,2RL,VEuPathDB,protein_coding_gene,76406705,76407655,NaN,+,NaN,LOC125763977,NaN,NaN,"glutathione S-transferase 1-like, transcript v..."


Another fairly simple case is when we know what the description of the gene is. For instance, if we are interested in ACE1, its description is "Acetylcholinesterase [Source:UniProtKB/TrEMBL;Acc:A0A182RZ85]".

In [14]:
af1.genome_features().query("description == 'Acetylcholinesterase [Source:UniProtKB/TrEMBL;Acc:A0A182RZ85]'")

,contig,source,type,start,end,score,strand,phase,ID,Parent,Note,description
38520,2RL,VEuPathDB,protein_coding_gene,19417355,19490903,NaN,+,NaN,LOC125763056,NaN,NaN,Acetylcholinesterase [Source:UniProtKB/TrEMBL;...


The "[Source:UniProtKB/TrEMBL;Acc:A0A182RZ85]" part makes it hard to remember and is thus problematic. It is possible, though not really simple, to dispense with it.

In [15]:
correct_id  = [i for i, d in enumerate(af1.genome_features()['description']) if type(d) == str if d.split(" ")[0] == 'Acetylcholinesterase']
af1.genome_features().loc[correct_id[0]]

contig                                                       2RL
source                                                 VEuPathDB
type                                         protein_coding_gene
start                                                   19417355
end                                                     19490903
score                                                        NaN
strand                                                         +
phase                                                        NaN
ID                                                  LOC125763056
Parent                                                       NaN
Note                                                         NaN
description    Acetylcholinesterase [Source:UniProtKB/TrEMBL;...
Name: 38520, dtype: object

Another way to find genes of interest is by looking at specific regions of the genome, for instance ones that are around signals of selection. For instance, if we look at the region "X:8445000-8455000", we find Cyp9k1.

In [19]:
af1.genome_features(region="X:8445000-8455000").query("type == 'protein_coding_gene'")

The most common way to find a gene, however, is to use [Vectorbase](https://vectorbase.org/vectorbase/app) and look for orthologs for known genes in other species. For instance, if one looks at the orthologs of AGAP004707 (VGSC in _An. gambiae_), one can find several proteins of the gene AFUN2_008728 of AfunGA1 which can be found in the region "3RL:44105643-44156624". If we look at this region, we find the gene VGSC for _An. funestus_.

In [20]:
af1.genome_features(region="3RL:44105643-44156624").query("type == 'protein_coding_gene'")

,contig,source,type,start,end,score,strand,phase,ID,Parent,Note,description
0,3RL,VEuPathDB,protein_coding_gene,44105643,44156624,NaN,-,NaN,LOC125769886,NaN,NaN,"sodium channel protein para, transcript varian..."


1. Sinka, M.E., Bangs, M.J., Manguin, S. et al. A global map of dominant malaria vectors. Parasites Vectors 5, 69 (2012). https://doi.org/10.1186/1756-3305-5-69
2. Msugupakulya, B.J., Urio, N.H., Jumanne, M. et al. Changes in contributions of different Anopheles vector species to malaria transmission in east and southern Africa from 2000 to 2022. Parasites Vectors 16, 408 (2023). https://doi.org/10.1186/s13071-023-06019-1
3. McCann RS, Ochomo E, Bayoh MN, Vulule JM, Hamel MJ, Gimnig JE, Hawley WA, Walker ED. Reemergence of Anopheles funestus as a vector of Plasmodium falciparum in western Kenya after long-term implementation of insecticide-treated bed nets. Am J Trop Med Hyg. 2014 Apr;90(4):597-604. doi: 10.4269/ajtmh.13-0614. Epub 2014 Jan 27. PMID: 24470562; PMCID: PMC3973499.
4. Ntabaliba W, Vavassori L, Stica C, Makungwa N, Odufuwa OG, Swai JK, Lekundayo R, Moore S. Life expectancy of Anopheles funestus is double that of Anopheles arabiensis in southeast Tanzania based on mark-release-recapture method. Sci Rep. 2023 Sep 22;13(1):15775. doi: 10.1038/s41598-023-42761-3. PMID: 37737323; PMCID: PMC10516982.
5. Ogola EO, Fillinger U, Ondiba IM, Villinger J, Masiga DK, Torto B, Tchouassi DP. Insights into malaria transmission among Anopheles funestus mosquitoes, Kenya. Parasit Vectors. 2018 Nov 6;11(1):577. doi: 10.1186/s13071-018-3171-3. PMID: 30400976; PMCID: PMC6219006.
6. Djouaka R, Akoton R, Tchigossou GM, Atoyebi SM, Irving H, Kusimo MO, Djegbe I, Riveron JM, Tossou E, Yessoufou A, Wondji CS. Mapping the distribution of Anopheles funestus across Benin highlights a sharp contrast of susceptibility to insecticides and infection rate to Plasmodium between southern and northern populations. Wellcome Open Res. 2016 Dec 14;1:28. doi: 10.12688/wellcomeopenres.10213.2. PMID: 28191507; PMCID: PMC5300096.
7. Dia I., Wamdaogo M., Ayala D., “Advances and Perspectives in the Study of the Malaria Mosquito Anopheles funestus” in Anopheles Mosquitoes - New Insights into Malaria Vectors, Manguin S., Ed. (InTech, 2013). 
8. Moiroux N, Gomez MB, Pennetier C, Elanga E, Djènontin A, Chandre F, Djègbé I, Guis H, Corbel V. Changes in Anopheles funestus biting behavior following universal coverage of long-lasting insecticidal nets in Benin. J Infect Dis. 2012 Nov 15;206(10):1622-9. doi: 10.1093/infdis/jis565. Epub 2012 Sep 10. PMID: 22966127.
9. Sougoufara S, Diédhiou SM, Doucouré S, Diagne N, Sembène PM, Harry M, Trape JF, Sokhna C, Ndiath MO. Biting by Anopheles funestus in broad daylight after use of long-lasting insecticidal nets: a new challenge to malaria elimination. Malar J. 2014 Mar 28;13:125. doi: 10.1186/1475-2875-13-125. PMID: 24678587; PMCID: PMC3973838.
10. Omondi S, Kosgei J, Musula G, Muchoki M, Abong'o B, Agumba S, Ogwang C, McDermott DP, Donnelly MJ, Staedke SG, Schultz J, Gutman JR, Gimnig JE, Ochomo E. Late morning biting behaviour of Anopheles funestus is a risk factor for transmission in schools in Siaya, western Kenya. Malar J. 2023 Nov 30;22(1):366. doi: 10.1186/s12936-023-04806-w. PMID: 38037026; PMCID: PMC10691009.
11. Nambunga IH, Ngowo HS, Mapua SA, Hape EE, Msugupakulya BJ, Msaky DS, Mhumbira NT, Mchwembo KR, Tamayamali GZ, Mlembe SV, Njalambaha RM, Lwetoijera DW, Finda MF, Govella NJ, Matoke-Muhia D, Kaindoa EW, Okumu FO. Aquatic habitats of the malaria vector Anopheles funestus in rural south-eastern Tanzania. Malar J. 2020 Jun 23;19(1):219. doi: 10.1186/s12936-020-03295-5. PMID: 32576200; PMCID: PMC7310514.
12. Jones, C.M., Lee, Y., Kitchen, A. et al. Complete Anopheles funestus mitogenomes reveal an ancient history of mitochondrial lineages and their distribution in southern and central Africa. Sci Rep 8, 9054 (2018). https://doi.org/10.1038/s41598-018-27092-y
13. The origin of our species. National History Museum, London, UK. https://www.nhm.ac.uk/discover/the-origin-of-our-species.html
14. Ayala D, Akone-Ella O, Kengne P, Johnson H, Heaton H, Collins J, Krasheninnikova K, Pelan S, Pointon DL, Sims Y, Torrance J, Tracey A, Uliano-Silva M, von Wyschetzki K, Wood J; DNA Pipelines collective; McCarthy S, Neafsey D, Makunin A, Lawniczak M. The genome sequence of the malaria mosquito, Anopheles funestus, Giles, 1900. Wellcome Open Res. 2023 Mar 27;7:287. doi: 10.12688/wellcomeopenres.18445.2. PMID: 36874567; PMCID: PMC9975407.